# Ex13

Consideremos o problema da metalurgia de alumínio, em que se deseja produzir 2.000 kg de uma liga de alumínio, a custo mínimo, pela mistura de diversas matérias-primas (minérios). Esta liga deve atender a requisitos de engenharia que especificam os máximos e mínimos de diversos elementos químicos que a compõem. Os custos das matérias-primas são:

| Matéria-prima | Mat1 | Mat2 | Mat3 | Mat4 | Mat5 | Mat6 | Mat7 |
|---------------|------|------|------|------|------|------|------|
| Custo         | 0,03 | 0,08 | 0,17 | 0,12 | 0,15 | 0,21 | 0,38 |

A composição dos minérios e a participação mínima/máxima de cada um dos elementos químicos nos 2.000 kg da liga são mostradas a seguir.


| Elemento | Mat1  | Mat2  | Mat3  | Mat4  | Mat5  | Al-puro | Si-puro | Mínimo | Máximo |
|----------|------|------|------|------|------|---------|---------|--------|--------|
| Fe       | 0,15 | 0,04 | 0,02 | 0,04 | 0,02 | 0,01    | 0,03    | -      | 60     |
| Cu       | 0,03 | 0,05 | 0,08 | 0,02 | 0,06 | 0,01    | -       | -      | 100    |
| Mn       | 0,02 | 0,04 | 0,01 | 0,02 | 0,02 | -       | -       | -      | 40     |
| Mg       | 0,02 | 0,03 | -    | -    | 0,01 | -       | -       | -      | 30     |
| Al       | 0,70 | 0,75 | 0,8 | 0,75  | 0,80 | 0,97    | -       | 1500     | -      |
| Si       | 0,02 | 0,06 | 0,08 | 0,12 | 0,02 | 0,01    | 0,97    | 250    | 300    |

Na tabela anterior temos, por exemplo, que Mat1 contém 15% de Ferro, 3% de Cobre, etc. Temos, ainda, que a liga a ser obtida (2.000 kg) deve conter, no máximo, 60 kg de Ferro, 100 kg de Cobre e que a quantidade de Silício deve estar entre 250 kg e 300 kg.

Quanto à disponibilidade de matéria-prima, os dados estão indicados a seguir na linha "Disponibilidade Máxima". A linha "Disponibilidade Mínima" refere-se à quantidade que se deseja forçar a entrar neste processo (por algum motivo, tal como liberação de espaço).


|          | Mat1 | Mat2 | Mat3 | Mat4 | Mat5 | Mat6   |Mat7   |
|----------|------|------|------|------|------|--------|--------|
| Disp. Mín.| -    | -    | 400  | 100  | -    | -      | -
| Disp. Máx.| 200  | 750  | 800  | 700  | 1500 | Infinito |Infinito |





### Modelagem Matemática

#### **Variáveis de decisão**
Definimos $x_i$ como a quantidade (kg) de cada matéria-prima $i$ utilizada na mistura, onde $i \in \{1, 2, 3, 4, 5, 6, 7\}$.

#### **Função objetivo**
Minimizar o custo total de produção da liga:

$$
\text{Minimize } Z = 0,03x_1 + 0,08x_2 + 0,17x_3 + 0,12x_4 + 0,15x_5 + 0,21x_6 + 0,38x_7
$$

#### **Restrições**
1. **Quantidade total da liga**:
$$
x_1 + x_2 + x_3 + x_4 + x_5 + x_6 + x_7 = 2000
$$

2. **Restrições de composição química**:

   - **Ferro**:
$$
0,15x_1 + 0,04x_2 + 0,02x_3 + 0,04x_4 + 0,02x_5 + 0,01x_6 + 0,03x_7 \leq 60
$$

   - **Cobre**:
$$
0,03x_1 + 0,05x_2 + 0,08x_3 + 0,02x_4 + 0,06x_5 + 0,01x_6 \leq 100
$$

   - **Manganês**:
$$
0,02x_1 + 0,04x_2 + 0,01x_3 + 0,02x_4 + 0,02x_5 \leq 40
$$

   - **Magnésio**:
$$
0,02x_1 + 0,03x_2 + 0x_3 + 0x_4 + 0,01x_5 \leq 30
$$

   - **Silício**:
$$
250 \leq 0,02x_1 + 0,06x_2 + 0,08x_3 + 0,12x_4 + 0,02x_5 + 0,01x_6 + 0,97x_7 \leq 300
$$

3. **Limites de disponibilidade de matéria-prima**:
$$
\begin{aligned}
0 \leq x_1 \leq 200 \\
0 \leq x_2 \leq 750 \\
400 \leq x_3 \leq 800 \\
100 \leq x_4 \leq 700 \\
0 \leq x_5 \leq 1500 \\
0 \leq x_6 \leq \infty \\
0 \leq x_7 \leq \infty \\
\end{aligned}
$$


In [1]:
import pyomo.environ as pyo

In [2]:
# Criação do modelo
model = pyo.ConcreteModel()

In [3]:

# Conjuntos
model.M = pyo.Set(initialize=["Mat1", "Mat2", "Mat3", "Mat4", "Mat5", "Mat6", "Mat7"])

model.E = pyo.Set(initialize=["Fe", "Cu", "Mn", "Mg", "Al", "Si"])

In [4]:
custo_material = {"Mat1": 0.03, "Mat2": 0.08, "Mat3": 0.17, "Mat4": 0.12, 
                  "Mat5": 0.15, "Mat6": 0.21, "Mat7": 0.38}
model.c_material = pyo.Param(model.M, initialize=custo_material)

# Composição química das matérias-primas
qtd_elementos = {
    ("Mat1", "Fe"): 0.15, ("Mat2", "Fe"): 0.04, ("Mat3", "Fe"): 0.02, ("Mat4", "Fe"): 0.04, ("Mat5", "Fe"): 0.02, ("Mat6", "Fe"): 0.01, ("Mat7", "Fe"): 0.03,
    ("Mat1", "Cu"): 0.03, ("Mat2", "Cu"): 0.05, ("Mat3", "Cu"): 0.08, ("Mat4", "Cu"): 0.02, ("Mat5", "Cu"): 0.06, ("Mat6", "Cu"): 0.01,
    ("Mat1", "Mn"): 0.02, ("Mat2", "Mn"): 0.04, ("Mat3", "Mn"): 0.01, ("Mat4", "Mn"): 0.02, ("Mat5", "Mn"): 0.02,
    ("Mat1", "Mg"): 0.02, ("Mat2", "Mg"): 0.03, ("Mat3", "Mg"): 0.00, ("Mat4", "Mg"): 0.00, ("Mat5", "Mg"): 0.01,
    ("Mat1", "Al"): 0.70, ("Mat2", "Al"): 0.75, ("Mat3", "Al"): 0.80, ("Mat4", "Al"): 0.75, ("Mat5", "Al"): 0.80, ("Mat6", "Al"): 0.97,
    ("Mat1", "Si"): 0.02, ("Mat2", "Si"): 0.06, ("Mat3", "Si"): 0.08, ("Mat4", "Si"): 0.12, ("Mat5", "Si"): 0.02, ("Mat6", "Si"): 0.01, ("Mat7", "Si"): 0.97,
}
model.q_material = pyo.Param(model.M, model.E, initialize=qtd_elementos, default=0)

# Limites de disponibilidade das matérias-primas
estoque_material = {"Mat1": 200, "Mat2": 750, "Mat3": 800, "Mat4": 700, "Mat5": 1500, "Mat6": float("inf"), "Mat7": float("inf")}
model.e_material = pyo.Param(model.M, initialize=estoque_material)

# Limites de presença de elementos químicos
limites_elementos = {"Fe": (0, 60), "Cu": (0, 100), "Mn": (0, 40), "Mg": (0, 30), "Al": (1500, float("inf")), "Si": (250, 300)}
model.l_elemento_min = pyo.Param(model.E, initialize={e: limites_elementos[e][0] for e in model.E})
model.l_elemento_max = pyo.Param(model.E, initialize={e: limites_elementos[e][1] for e in model.E})


In [5]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------

model.x = pyo.Var(model.M, domain=pyo.NonNegativeReals)

In [6]:

# ------------------------------
# Função Objetivo
# ------------------------------
def fo(model):
    return sum(model.x[m] * model.c_material[m] for m in model.M)

model.objetivo = pyo.Objective(rule=fo, sense=pyo.minimize)


In [7]:
# ------------------------------
# Restrições
# ------------------------------

# 1. Limite total da liga
def liga_total_rule(model):
    return sum(model.x[m] for m in model.M) == 2000

model.liga_total = pyo.Constraint(rule=liga_total_rule)


# 2. Restrições de composição química
def elemento_min_rule(model, e):
    return sum(model.x[m] * model.q_material[m, e] for m in model.M) >= model.l_elemento_min[e]

def elemento_max_rule(model, e):
    return sum(model.x[m] * model.q_material[m, e] for m in model.M) <= model.l_elemento_max[e]

model.limite_elemento_min = pyo.Constraint(model.E, rule=elemento_min_rule)
model.limite_elemento_max = pyo.Constraint(model.E, rule=elemento_max_rule)

# 3. Limites de disponibilidade das matérias-primas
def estoque_rule(model, m):
    return model.x[m] <= model.e_material[m]

model.limite_estoque = pyo.Constraint(model.M, rule=estoque_rule)



In [8]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex13.lp", io_options={"symbolic_solver_labels": True})

('ex13.lp', 1848009184832)

In [9]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.objetivo)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)

LP   has 20 rows; 7 cols; 82 nonzeros
Coefficient ranges:
  Matrix [1e-02, 1e+00]
  Cost   [3e-02, 4e-01]
  Bound  [0e+00, 0e+00]
  RHS    [3e+01, 2e+03]
Presolving model
8 rows, 7 cols, 48 nonzeros  0s
Dependent equations search running on 1 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
7 rows, 7 cols, 41 nonzeros  0s
Presolve : Reductions: rows 7(-13); columns 7(-0); elements 41(-41)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 3(687.5) 0s
          7     2.9621660650e+02 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 7
Objective value     :  2.9621660650e+02
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.00

Status do solver: ok
Condição de terminação: optimal
Valor da Função Objetivo: R$ 296,22


In [10]:
# Exibindo os valores das variáveis de decisão
print("Resultados do Modelo:")
for m in model.M:
    valor = pyo.value(model.x[m])
    print(f"Quantidade de {m}: {valor:.2f} kg")



Resultados do Modelo:
Quantidade de Mat1: 0.00 kg
Quantidade de Mat2: 665.34 kg
Quantidade de Mat3: 490.25 kg
Quantidade de Mat4: 424.19 kg
Quantidade de Mat5: 0.00 kg
Quantidade de Mat6: 299.64 kg
Quantidade de Mat7: 120.58 kg
